In [1]:
'''
DESCRIPTION 
    Utility function that chops a sequence into several reads with bounded random lengths that 
    have a bounded random overlap
INPUT
    sequence       | a sequence of characters that will be divided into overlapping subsequences
    min_subseq_len | the shortest length a subsequence can have
    max_subseq_len | the longest length a subsequence can have
    min_overlap    | the shortest overlap two subsequences can share
    max_overlap    | the longest overlap two subsequences can share
    circularize    | boolean indicating whether to add a random amount of the end of the sequence
                   | to the beginning and vice versa
    seed           | random seed for the random function for reproducibility
OUTPUT
    A list of overlapping reads of random bounded size which share a bounded random amount of
    overlap
'''
def generate_reads(sequence,min_subseq_len,max_subseq_len,min_overlap,max_overlap,min_coverage=None,circularise=False,seed=None):
    import random

    random.seed(seed)
    if circularise: sequence = sequence[-random.randint(min_overlap,max_overlap):] + sequence + sequence[:random.randint(min_overlap,max_overlap)]
    reads = []
    while 1: 
        start = 0
        end = random.randint(min_subseq_len,max_subseq_len)
        reads += [sequence[start:end]]
        while end < len(sequence):
            start = random.randint(end-max_overlap,end-min_overlap)
            if (len(sequence) - start)/max_subseq_len < 2:
                if (len(sequence) - start)/max_subseq_len < 1:
                    end = len(sequence)
                else:
                    a = 0
                    while (len(sequence) - start)/(min_subseq_len+a) > 2: a+=1
                    end = random.randint(start+min_subseq_len+a,start+max_subseq_len) 
            else: end = random.randint(start+min_subseq_len,start+max_subseq_len) 
            reads += [sequence[start:end]]
        if min_coverage is None or len(set(reads))*(sum(len(read) for read in set(reads))/len(set(reads)))/len(sequence) >= min_coverage: return list(set(reads))

'''
DESCRIPTION 
    Utility function that creates a random sequence containing only the letters A, T, G, and C
INPUT
    n          | the length of the sequence
    palindrome | a boolean indicating whether the sequence must be a palidrome or not
    seed       | random seed for the random function for reproducibility
OUTPUT
    A random sequence of length n
'''
def generate_genome_sequence(n,palindrome=False,seed=None):
    import random,math
    
    random.seed(seed)
    nucleotides = {1:'A',2:'C',3:'G',4:'T'}
    seq = ''
    if palindrome: n = math.ceil(n/2)
    for _ in range(n):
        seq += nucleotides[random.randint(1,4)]
    if palindrome: seq += ''.join(reversed(seq[:int(n-math.fmod(n,2))]))
    return seq

# Sequitur

In [8]:
# ! pip install networkx
import networkx as nx
from networkx import bipartite

In [288]:
def build_suffix_array(reads):
    suf_arr = []
    for read in reads:
        read += '$' + str(reads.index(read))
        for i in range(len(read)):
            if len(read[i:]) < 4: continue 
            suf_arr += [read[i:]]
    suf_arr.sort()
    suf_arr_ind = []
    for s in range(len(suf_arr)):
        suf_arr_ind += [int(suf_arr[s][-1])]
        suf_arr[s] = suf_arr[s][:-1]
    return suf_arr,suf_arr_ind

def build_bipartite_graph(reads,suf_arr=None,suf_arr_ind=None):
    if suf_arr is None or suf_arr_ind is None: suf_arr,suf_arr_ind = build_suffix_array(reads)
    B = nx.Graph()
    for read in reads:
        i = suf_arr.index(read+'$') - 1
        while read.startswith(suf_arr[i][:-1]):
            B.add_edge('^' + reads[suf_arr_ind[i]],read + '$',weight=len(suf_arr[i][:-1]))
            B.add_edge('^' + reads[suf_arr_ind[i]],read + '$',weight=len(suf_arr[i][:-1]))
            i -= 1
    return B

def find_path(graph,draw=False):
    G_v = nx.DiGraph()
    for p,s in nx.maximal_matching(G):
        G_v.add_edge(p[1:],s[:-1],weight=nx.get_edge_attributes(G,"weight")[(p,s)])
    if draw: nx.draw(G_v)
    while not nx.is_connected(G_v.to_undirected()):
        add_edges = []
        remove_edges = []
        for node,degree in G_v.in_degree():
            if degree == 0:
                edges = list(G.edges(node+'$',data="weight"))
                edges.sort(key=lambda x: x[-1])
                remove_edges += G_v.out_edges(edges[0][1][1:])
                add_edges += [(edges[0][1][1:],edges[0][0][:-1],{'weight':edges[0][2]})]
        G_v.remove_edges_from(remove_edges)
        G_v.add_edges_from(add_edges)
        if draw: nx.draw(G_v)
    return G_v

def sequitur(path):
    while path.number_of_nodes() > 1:
        path.add_node(list(path.edges)[0][0] + list(path.edges)[0][1][list(path.edges(data='weight'))[0][2]:])
        predecessors = list(path.predecessors(list(path.edges)[0][0]))
        for pre in predecessors:
            path.add_edge(pre,list(path.edges)[0][0] + list(path.edges)[0][1][list(path.edges(data='weight'))[0][2]:],weight=path.edges[pre,list(path.edges)[0][0]]['weight'])
            path.remove_edge(pre,list(path.edges)[0][0])
        successors = list(path.successors(list(path.edges)[0][1]))
        for suc in successors:
            path.add_edge(list(path.edges)[0][0] + list(path.edges)[0][1][list(path.edges(data='weight'))[0][2]:],suc,weight=path.edges[list(path.edges)[0][1],suc]['weight'])
            path.remove_edge(list(path.edges)[0][1],suc)
        path.remove_nodes_from(list(path.edges)[0])
    return list(path.nodes())[0]

In [282]:
sequence = 'betty_bought_butter_the_butter_was_bitter_betty_bought_better_butter_to_make_the_bitter_butter_better'
reads = ['betty_bought_butter_th',
                        'tter_the_butter_was_',
                              'he_butter_was_bitter_',
                                         'as_bitter_betty_bought',
                                                     'tty_bought_better_butter_t',
                                                                     'r_butter_to_make_the_',
                                                                                   'ke_the_bitter_butter_better']

G = build_bipartite_graph(reads)

In [283]:
path = find_path(G)

In [289]:
sequitur(path)

'betty_bought_butter_the_butter_was_bitter_betty_bought_better_butter_to_make_the_bitter_butter_better'

In [ ]:
sequence = 'you say hello world, i bellow go to hell'
reads = ['you say hel',
            ' say hello wo',
                    'lo world, i be',
                          'ld, i bellow go t',
                                    'ow go to hell']
sequitur = Sequitur(reads,assemble=True)
sequitur.sequence == sequence if type(sequitur.sequence) is str else all(s in sequence for s in sequitur.sequence)

In [ ]:
sequence = 'she_sells_sea_shells_on_the_sea_shore'
reads = ['she_sells_s',
               'lls_sea_shel',
                    'ea_shells_o',
                       'shells_on_the_s',
                                  'he_sea_s',
                                      'ea_shore']
sequitur = Sequitur(reads,assemble=True)
sequitur.sequence == sequence if type(sequitur.sequence) is str else all(s in sequence for s in sequitur.sequence)

In [ ]:
successes = 0
n = 1
for seed in range(n):   
    # sequence = generate_genome_sequence(10000,seed=seed)
    # reads = generate_reads(sequence,250,500,50,100,seed=seed)
    sequitur = Sequitur(reads,assemble=True)
    s = '| Seed: ' + str(seed) + ' | '
    if type(sequitur.sequence) is str and sequitur.sequence == sequence:
        s+='SUC | ' + str(sequitur.sequence) + ' == ' + sequence
        successes+=1
    elif type(sequitur.sequence) is list and all(s in sequence for s in sequitur.sequence):
        s+='PAR | ' + str(sequitur.sequence) + ' ~~ ' + sequence
        successes+=0.5
    else: s+='FAI | ' + sequitur.sequence + ' != ' + sequence
    print(s)
    print('-----------------------------------------')
print('ACCURACY: '+str((successes/n)*100)+'%')

    SUC: returns the target sequence fully reconstructed
    PAR: returns contigs all of which exist in the target sequence (consider coverage?)
    FAI: returns a full sequence that is incorrectly reconstructed or a set of contigs where at least one is not found in the target sequence

In [ ]:
sequence = generate_genome_sequence(10000,seed=0)
reads = generate_reads(sequence,250,500,50,100,seed=0)
reads

In [ ]:
sequitur = Sequitur(reads,assemble=True)
sequitur.sequence,sequence

In [ ]:
sequitur.sequence[sequitur.sequence.find('TTAGTTGTGCCGCAGCGAAGTA'):] in sequence,\
sequitur.sequence[:sequitur.sequence.find('TTAGTTGTGCCGCAGCGAAGTA')] in sequence

In [ ]:
sequence = generate_genome_sequence(10000,palindrome=True,seed=0)
reads = generate_reads(sequence,250,500,50,100,seed=0)
reads

In [ ]:
sequitur = Sequitur(reads,assemble=True)
sequitur.sequence,sequence

In [ ]:
#! pip install Bio
from Bio import SeqIO, Seq

In [ ]:
for record in SeqIO.parse("data/input/Raphanus sativus_NC_018551.1.fasta",'fasta'):
    sequence = record.seq
    # print(record)
    # break
reads = generate_reads(sequence,250,250,50,50,min_coverage=None,seed=0)   